In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
from python_tools import file_utils as filu
from python_tools import module_utils as modu
from python_tools import inspect_utils as iu
from python_tools import package_utils as pku
from python_tools import pathlib_utils as plu

# Complete process of cleaning module

In [ ]:

"""
Purpose: Take an unordered module file and to clean it up
so avoids circular imports that could result in errors. And
also make it look professional


---- old ----
1) Get all of the modules used
2) Divide the modules into those referenced by outside packages and those not
3) Get all of the member variables
4) Get all of the doc strings
5) Arrange in the following order:
--------------


1) Run the replacement to put the right qualifiers in front of a module 
using the right package
2) Get all the docstrings and replace in the file with nothing
3) Find all of the module at the beginning of the line
4) Determine the package that each module came from (if it did have one)
5) Determine the parent module (if any)
6) Put the correct prefix in front of module if not already have it
- and add the relative one to those inside current package

--------------------------------------------
7) Why can't go collect current variables?
- because there could be lists  that screw up the regex
* the variables may sometimes depend on another class (but handle those on case by case)
--------------------------------------------

8) Replace the docstrng and the modules (at the begginning of line) with empty strings
9) write the data in following order
- at beginning: non-referencing modules, doc strings
- at end: referencing modules

non-referencing modules
#local veraibles
doc strings
rest of code
referencing modules
itself

-- things going to think about: 
those not at the start of the line --> do not want using the dot operator
for relative (other than that everything else should be the same)
"""

In [4]:
filepath = "/python_tools/python_tools/networkx_utils.py"
verbose = True

In [ ]:
# 1) Run replacement to put right qualifiers in front of something

In [40]:
from python_tools import file_utils as filu
filu.search_directory_files_for_str(
    "/python_tools/python_tools/",
    search_str="tqdm_utils import tqdm"
)

tqdm_utils import tqdm found in /python_tools/python_tools/networkx_utils.py
    Wasn't able to read /python_tools/python_tools/numpy_utils.py
    Wasn't able to read /python_tools/python_tools/ipyvolume_utils.py
tqdm_utils import tqdm found in /python_tools/python_tools/pandas_utils.py
tqdm_utils import tqdm found in /python_tools/python_tools/system_utils.py


['/python_tools/python_tools/networkx_utils.py',
 '/python_tools/python_tools/pandas_utils.py',
 '/python_tools/python_tools/system_utils.py']

In [57]:
directory = "/python_tools/python_tools/"
modules = pku.module_names_from_directories(
    directory
)

modules_dir = modu.find_import_modules_in_file(
    filepath,
    modules = modules,
    beginning_of_line = False
    )
modules_dir

['from . import string_utils as stru',
 'from .tqdm_utils import tqdm',
 'from python_tools import general_utils as gu',
 'from python_tools import tqdm_utils as tqu',
 'from . import numpy_utils as nu',
 'from python_tools import numpy_utils as nu',
 'from python_tools import regex_utils as ru',
 'from python_tools import system_utils as su',
 'from python_tools import string_utils as stru',
 'from . import networkx_utils as xu',
 'from python_tools import pandas_utils as pu',
 'from . import tqdm_utils as tqu',
 'from . import general_utils as gu',
 'from . import pandas_utils as pu',
 'from . import regex_utils as ru',
 'from python_tools import networkx_utils as xu']

In [ ]:
"""
Need to handle the dot differently
"""


In [55]:
modules_dir = modu.find_import_modules_in_file(
    filepath,
    modules = modules,
    )
modules_dir

['from . import string_utils as stru',
 'from .tqdm_utils import tqdm',
 'from . import numpy_utils as nu',
 'from . import networkx_utils as xu',
 'from . import tqdm_utils as tqu',
 'from . import general_utils as gu',
 'from . import pandas_utils as pu',
 'from . import regex_utils as ru']

# 1) Get all of the modules used

In [38]:
modules_used = modu.find_import_modules_in_file(
    filename=filepath,
    pattern = pattern_import,
    verbose = verbose,
    unique = True
)
modules_used

# of matches (unique = True) = 21


['from .tqdm_utils import tqdm',
 'import itertools',
 'import pandas as pd',
 'from . import general_utils as gu',
 'import networkx.classes.function as cls_func',
 'import numpy as np',
 'from . import string_utils as stru',
 'import pydot',
 'from . import pandas_utils as pu',
 'from . import regex_utils as ru',
 'from copy import deepcopy',
 'from networkx.classes.function import path_weight as pw',
 'import networkx as nx',
 'import time',
 'import matplotlib.pyplot as plt',
 'import random',
 'import copy',
 'from networkx.drawing.nx_pydot import graphviz_layout',
 'from . import networkx_utils as xu',
 'from . import tqdm_utils as tqu',
 'from . import numpy_utils as nu']

# 2) Divide modules into referenced outside of package and those not

In [ ]:
modules_directory = [
    "/meshAfterParty/meshAfterParty/",
    "/python_tools/python_tools/",
    "/graph_tools/graph_tools/",
    "/neuron_morphology_tools/neuron_morphology_tools/",
    "/pytorch_tools/pytorch_tools/",
]


#modu.prefix_module_imports_in_files

In [37]:
directory = "/python_tools/python_tools/"

modules = modu.modules_from_directory(
    directory = directory,
    verbose = verbose
)

pattern_import = modu.import_pattern_str(
    #modules = modules,
    verbose = verbose
)
py_tools_imports = modu.find_import_modules_in_file(
    filename=filepath,
    pattern = pattern_import,
    verbose = verbose,
    unique = False
)
py_tools_imports

# of modules = 36
import_str = (?:\A|\n)((?:import [a-zA-Z._]+ as [a-zA-Z._]+)|(?:from [a-zA-Z._]+ import [a-zA-Z._]+ as [a-zA-Z._]+)|(?:import [a-zA-Z._]+)|(?:from [a-zA-Z._]+ import [a-zA-Z._]+))
# of matches (unique = False) = 21


['from . import general_utils as gu',
 'from . import numpy_utils as nu',
 'from . import pandas_utils as pu',
 'from . import regex_utils as ru',
 'from . import string_utils as stru',
 'from . import tqdm_utils as tqu',
 'from .tqdm_utils import tqdm',
 'from copy import deepcopy',
 'from networkx.classes.function import path_weight as pw',
 'from networkx.drawing.nx_pydot import graphviz_layout',
 'import copy',
 'import itertools',
 'import matplotlib.pyplot as plt',
 'import networkx as nx',
 'import networkx.classes.function as cls_func',
 'import numpy as np',
 'import pandas as pd',
 'import pydot',
 'import random',
 'import time',
 'from . import networkx_utils as xu']

# 3) Get all the member variables

In [12]:
from python_tools import networkx_utils as xu
iu.global_vars(xu,verbose = True)

# of global variables = 3


['downstream_name', 'node_id_default', 'upstream_name']

In [ ]:
"""
Can we detect member variables without the module? no not necessarily
"""

# 4) Get all docstrings

In [6]:
multi_line_comm = modu.multiline_str(
    filepath = filepath,
    verbose = verbose,
    return_text = True
)

# of multi-line strings = 3
